In [27]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [28]:
import logging
from pathlib import Path

import altair as alt
import holoviews as hv
import hvplot.polars  # noqa
import numpy as np
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.eda import detrend_tonic_component
from src.features.scaling import scale_min_max
from src.features.utils import to_describe
from src.log_config import configure_logging
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
    plot_max_correlations_by_participant,
)

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [29]:
db = DatabaseManager()
with db:
    df = db.get_table(
        "Merged_and_Labeled_Data",
        exclude_trials_with_measurement_problems=True,
    )

# Add detrended EDA
# different paradigms: statistics and statistical learning / non-causal and causal
df = detrend_tonic_component(df)

# Rename columns
df = df.drop(["heart_rate", "pupil_mean"]).rename(
    {
        "rating": "pain_rating",
        # NOTE: using the non-causal data columns here
        "heart_rate_exploratory": "heart_rate",
        "pupil_mean_exploratory": "pupil_diameter",
        # eda tonic and phasic are already non-causally transformed
    }
)

In [30]:
# Remove first 20 seconds
df = df.filter(col("normalized_timestamp") >= 20 * 1000)
df

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heart_rate,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_r_exploratory,pupil_l_exploratory,pupil_diameter,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals,eda_tonic_detrended
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16,f64
1,1,1,314198.9762,0.463265,0.87188,0.743199,0.743294,-0.000095,1375.838609,-1.67065,59.85067,4.577485,4.825832,4.933592,4.448417,4.933559,4.448429,4.690994,0.00087,0.000512,0.000438,0.00001,0.000085,20000.0,396,1,0,0,1,0,0,1,0,-0.000316
1,1,1,314298.9762,0.463265,0.824514,0.74323,0.743297,-0.000067,1349.613188,-0.775775,60.223816,4.590687,4.831406,4.934665,4.446768,4.934707,4.446755,4.690731,0.000702,0.000403,0.000291,2.9873e-7,0.000041,20100.0,396,1,0,0,1,0,0,1,0,-0.000313
1,1,1,314398.9762,0.463265,0.815,0.743276,0.7433,-0.000023,1325.324829,-0.88756,60.618429,4.593487,4.862389,4.930151,4.476284,4.930121,4.476294,4.703207,0.000663,0.000418,0.000106,0.0,0.000058,20200.0,396,1,0,0,1,0,0,1,0,-0.000311
1,1,1,314498.9762,0.463265,0.815,0.743297,0.743303,-0.000006,1297.487842,-1.209588,61.04456,4.555876,4.885492,4.907607,4.518567,4.907623,4.518566,4.713094,0.000739,0.00045,0.00013,0.0,0.00005,20300.0,396,1,0,0,1,0,0,1,0,-0.000307
1,1,1,314598.9762,0.463265,0.815,0.743265,0.743306,-0.000041,1291.534487,-0.726068,61.483515,4.497064,4.882886,4.888406,4.528388,4.888402,4.528379,4.708391,0.000657,0.000379,0.000132,0.0,0.000058,20400.0,396,1,0,0,1,0,0,1,0,-0.000304
1,1,1,314698.9762,0.463265,0.815,0.743194,0.74331,-0.000117,1371.283878,-1.318697,61.95022,4.464745,4.862969,4.882415,4.528865,4.882412,4.528885,4.705649,0.000824,0.000453,0.000169,0.0,0.00005,20500.0,396,1,0,0,1,0,0,1,0,-0.0003
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
516,12,50,2.6956e6,0.031312,0.0,26.688608,26.716225,0.003169,1323.813266,-5.036073,59.394254,5.166699,4.766597,5.029708,5.52614,5.029708,5.526104,5.277906,0.000465,0.001124,0.007081,0.00022,0.004168,179500.0,806,1,2580,1548,0,0,0,0,0,-0.071205
516,12,50,2.6957e6,0.030874,0.0,26.68499,26.716212,0.000389,1244.871959,44.337204,58.82839,5.026438,4.683481,5.016028,5.473471,5.016028,5.473472,5.24475,0.000698,0.000846,0.007044,0.000348,0.004289,179600.0,806,1,2580,1548,0,0,0,0,0,-0.070247


In [31]:
scale_min_max(df).hvplot(
    x="timestamp",
    y=[
        "eda_raw",
        "eda_phasic",
        "eda_tonic",
        # "pupil",
        # "rating",
        "temperature",
        # "pupil_r_raw",
        # "pupil_l_raw",
    ],
    groupby=["participant_id", "trial_number"],
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'768585c2-f27a-4836-bbe6-4746714e3fd4': {'version…

# Temperature

In [32]:
COLORS = {
    "temperature_pain_rating_corr": "#1f77b4",
    "temperature_pupil_diameter_corr": "#ff7f0e",
    "temperature_eda_tonic_corr": "#d62728",
    "temperature_eda_phasic_corr": "#8a2be2",
    "temperature_heart_rate_corr": "#2ca02c",
}


## Temperature / Rating

In [33]:
col1, col2 = "temperature", "pain_rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
rating = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=False
)
rating


alt.LayerChart(...)

In [34]:
corr_by_participant.describe()

statistic,participant_id,participant_id_temperature_pain_rating_corr_mean,participant_id_temperature_pain_rating_corr_ci_lower,participant_id_temperature_pain_rating_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",25.928571,0.775713,0.739026,0.807456
"""std""",14.68763,0.045116,0.054968,0.038745
"""min""",1.0,0.686459,0.627146,0.725711
"""25%""",13.0,0.744626,0.700885,0.786013
"""50%""",26.0,0.787325,0.752075,0.814521
"""75%""",38.0,0.80805,0.776999,0.830903
"""max""",50.0,0.867533,0.853177,0.880576


## Temperature / Pupil

In [35]:
col1, col2 = "temperature", "pupil_diameter"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
pupil = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=-False, color_map=COLORS
)
pupil

alt.LayerChart(...)

In [36]:
corr_by_participant.describe()

statistic,participant_id,participant_id_temperature_pupil_diameter_corr_mean,participant_id_temperature_pupil_diameter_corr_ci_lower,participant_id_temperature_pupil_diameter_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",25.928571,0.26357,0.147719,0.372418
"""std""",14.68763,0.210145,0.22461,0.19693
"""min""",1.0,-0.174514,-0.278513,-0.118857
"""25%""",13.0,0.125258,-0.009725,0.256528
"""50%""",26.0,0.280388,0.145873,0.412657
"""75%""",38.0,0.447849,0.322228,0.548107
"""max""",50.0,0.631317,0.546054,0.703635


## Temperature / EDA Tonic

In [37]:
# Note that correlation values are slightly higher if we'd used detrended EDA tonic
# components (see eda.py)
# EDA tonic detrended > EDA tonic > EDA raw > EDA phasic
col1, col2 = "temperature", "eda_tonic"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
# plot_correlations_by_trial(corr_by_trial, col1, col2)
eda_tonic = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=-False, color_map=COLORS
)
eda_tonic


alt.LayerChart(...)

In [38]:
corr_by_participant.describe()

statistic,participant_id,participant_id_temperature_eda_tonic_corr_mean,participant_id_temperature_eda_tonic_corr_ci_lower,participant_id_temperature_eda_tonic_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",25.928571,0.216795,0.086044,0.338928
"""std""",14.68763,0.207387,0.208802,0.202433
"""min""",1.0,-0.094434,-0.203011,0.016438
"""25%""",13.0,0.051318,-0.078004,0.159491
"""50%""",26.0,0.156169,0.040391,0.319127
"""75%""",38.0,0.373497,0.229845,0.501141
"""max""",50.0,0.684461,0.591577,0.761265


## Temperature / EDA Phasic

In [39]:
col1, col2 = "temperature", "eda_phasic"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
# plot_correlations_by_trial(corr_by_trial, col1, col2)
eda_phasic = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=-False, color_map=COLORS
)
eda_phasic

alt.LayerChart(...)

In [40]:
corr_by_participant.describe()

statistic,participant_id,participant_id_temperature_eda_phasic_corr_mean,participant_id_temperature_eda_phasic_corr_ci_lower,participant_id_temperature_eda_phasic_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",25.928571,0.07873,0.037873,0.119372
"""std""",14.68763,0.057321,0.065947,0.05117
"""min""",1.0,-0.058635,-0.134986,-0.007413
"""25%""",13.0,0.048436,-0.001593,0.088374
"""50%""",26.0,0.080869,0.044934,0.124931
"""75%""",38.0,0.114724,0.083222,0.15154
"""max""",50.0,0.189497,0.160349,0.218316


## Temperature / Heartrate

In [41]:
col1, col2 = "temperature", "heart_rate"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
heart_rate = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=-False, color_map=COLORS
)
heart_rate

alt.LayerChart(...)

In [42]:
corr_by_participant.describe()

statistic,participant_id,participant_id_temperature_heart_rate_corr_mean,participant_id_temperature_heart_rate_corr_ci_lower,participant_id_temperature_heart_rate_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",25.928571,0.123188,0.040466,0.204246
"""std""",14.68763,0.158025,0.162589,0.15362
"""min""",1.0,-0.153126,-0.263415,-0.087913
"""25%""",13.0,-0.021189,-0.10255,0.056045
"""50%""",26.0,0.117714,0.033127,0.190847
"""75%""",38.0,0.221065,0.135985,0.302461
"""max""",50.0,0.531,0.477288,0.58076


## Temperature / All

In [43]:
(
    heart_rate
    + eda_tonic
    + eda_phasic
    + pupil
    + rating  # put the plot with the color_map first,
).configure_axis(
    grid=True,
).properties(
    title="Correlations with Temperature",
).configure_legend(labelFontSize=14)

alt.LayerChart(...)

# Rating

In [44]:
COLORS = {
    "pain_rating_temperature_corr": "#1f77b4",
    "pain_rating_pupil_diameter_corr": "#ff7f0e",
    "pain_rating_eda_tonic_corr": "#d62728",
    "pain_rating_eda_phasic_corr": "#8a2be2",
    "pain_rating_heart_rate_corr": "#2ca02c",
}

## Rating / Temperature

In [45]:
col1, col2 = "pain_rating", "temperature"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
temperature = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=False, color_map=COLORS
)
temperature


alt.LayerChart(...)

## Rating / Pupil

In [46]:
col1, col2 = "pain_rating", "pupil_diameter"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
pupil = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=False, color_map=COLORS
)
pupil

alt.LayerChart(...)

## Rating / EDA Tonic

In [47]:
col1, col2 = "pain_rating", "eda_tonic"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
eda_tonic = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=False, color_map=COLORS
)
eda_tonic

alt.LayerChart(...)

## Rating / Phasic EDA

In [48]:
col1, col2 = "pain_rating", "eda_phasic"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
eda_phasic = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=False, color_map=COLORS
)
eda_phasic

alt.LayerChart(...)

## Rating / Heart Rate

In [49]:
col1, col2 = "pain_rating", "heart_rate"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
heart_rate = plot_correlations_by_participant(
    corr_by_participant, col1, col2, with_config=False, color_map=COLORS
)
heart_rate


alt.LayerChart(...)

## Rating / All

In [50]:
(
    heart_rate
    + eda_tonic
    + eda_phasic
    + pupil
    + temperature  # put the plot with the color_map first,
).configure_axis(
    grid=True,
).properties(
    title="Correlations with Rating",
).configure_legend(labelFontSize=14)

alt.LayerChart(...)

# Maximum correlation values only

In [51]:
col1, col2 = "temperature", "pupil"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, col1, col2, "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, col1, col2)
pupil = plot_max_correlations_by_participant(
    corr_by_trial, col1, col2, with_config=-False
)
pupil

ColumnNotFoundError: pupil

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["trial_id", "trial_number", "participant_id", "timestamp", ...]; PROJECT */35 COLUMNS

In [ ]:
pupil + heart_rate

alt.LayerChart(...)